
 # Projet de Machine Learning
    
 #  Membres du groupe:
   
    
        [=> ANABEBOU Issaka Tigabitoua
        [=> KABORE Moumini
        [=> SOURABIE Kikin Abdoul Kader


# 0. Import des bibiliothèques

In [19]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator # Pour l'évaluation  du modèle de régression linéaire
from pyspark.ml.regression import DecisionTreeRegressor # Pour l'arbre de décision
from pyspark.ml.classification import RandomForestClassifier # Pour Random forest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F
from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 1. Chargement des données

In [69]:
# 1.1. Création d'une session 

spark = SparkSession.builder.master('local').appName('Projetc-G2').appName('df').getOrCreate()

# 1.2. Indexation du répertoire de travail

os.chdir("C:/Users/munkab/Documents/TEELGRE/5.Connaissances/M9. Master-FD-IA-UV/M1/12.Machine_Learning/ML-Projet-Test-Classification/ML-Test-Classification")

# 1.3. Chargement de la base de données

ccdefault = spark.read.format('csv').options(header=True, inferSchema=True).load("data/ccdefault.csv")

# 2. Analyse expliratoire de la base de données

In [15]:


# 2.1. Résumé statistiques.

ccdefault.select(ccdefault.columns).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
ID,30000,15000.5,8660.398374208891,1,30000
LIMIT_BAL,30000,167484.32266666667,129747.66156720246,10000,1000000
SEX,30000,1.6037333333333332,0.4891291960902602,1,2
EDUCATION,30000,1.8531333333333333,0.7903486597207269,0,6
MARRIAGE,30000,1.5518666666666667,0.5219696006132467,0,3
AGE,30000,35.4855,9.217904068090155,21,79
PAY_0,30000,-0.0167,1.1238015279973335,-2,8
PAY_2,30000,-0.13376666666666667,1.1971859730345495,-2,8
PAY_3,30000,-0.1662,1.1968675684465686,-2,8


In [11]:
# 2.1. Vérification de l'existence des valeurs manquantes.

for c in ccdefault.columns:
    count = ccdefault.filter(c+" is NULL" or c+" is ''" or c+" is NaN" or c+" is Null").count()
    if count > 0:
        print(c+" possèdent "+str(count)+" variables manquantes")
        
# NB: L'exécution du script ne signale auunes valeurs manquantes.
        
# 2.2. La corrélation entre les diffétentes variables

assembler = VectorAssembler().setInputCols(ccdefault.columns).setOutputCol("features")

assembler.transform(ccdefault).toPandas().head(5)


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT,features
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,689,0,0,0,0,1,"[1.0, 20000.0, 2.0, 2.0, 1.0, 24.0, 2.0, 2.0, ..."
1,2,120000,2,2,2,26,-1,2,0,0,...,3455,3261,0,1000,1000,1000,0,2000,1,"[2.0, 120000.0, 2.0, 2.0, 2.0, 26.0, -1.0, 2.0..."
2,3,90000,2,2,2,34,0,0,0,0,...,14948,15549,1518,1500,1000,1000,1000,5000,0,"[3.0, 90000.0, 2.0, 2.0, 2.0, 34.0, 0.0, 0.0, ..."
3,4,50000,2,2,1,37,0,0,0,0,...,28959,29547,2000,2019,1200,1100,1069,1000,0,"[4.0, 50000.0, 2.0, 2.0, 1.0, 37.0, 0.0, 0.0, ..."
4,5,50000,1,2,1,57,-1,0,-1,0,...,19146,19131,2000,36681,10000,9000,689,679,0,"[5.0, 50000.0, 1.0, 2.0, 1.0, 57.0, -1.0, 0.0,..."


# 3. Entrainent de modèle pour la prédiction de la variable cible.

In [70]:
# 3.0. Préparation de la base de données pour l'exécution des algorithmes 


# 3.0.1. Normalisation des données des variables de la base de données excéptée la variable cible (celle à prédire)

assembler = VectorAssembler(inputCols=[c for c in ccdefault.columns if c!= 'DEFAULT'], outputCol="features")
ccdefault = assembler.transform(ccdefault)


# 3.0.1. Echantillonnage des bases des bases d'entraitement et de test
#        NB: train_df : 70% de la base de données
#            test_df:   30% de la base de données

train_df, test_df = ccdefault.randomSplit([0.7, 0.3])


# 3.0.1. Exclusion de la variable "features" des bases d'entrainement et de test afin de pouvoir les utiliser pour les 
#        algoritmes de Linear Regression(LR) et Decision Tree (DT). 

train_df_lr_dt = train_df.drop("features")
test_df_lr_dt = test_df.drop("features")

In [71]:
# 3.1. Régression linéaire
#      --------------------


# 3.1.1. Pramétrage du modèle de regression linéaire

lr = LinearRegression(labelCol='DEFAULT')

pipeline = Pipeline(stages=[assembler, lr])


# 3.1.2. Entrainent  du modèle de régression linéaire

model = pipeline.fit(train_df_lr_dt)


# 3.1.3. Prédiction du modèle de régression linéaire

predictions = model.transform(test_df_lr_dt)

# 3.1.4. Evaluation du modèle de régression linéaire

eval = RegressionEvaluator(labelCol='DEFAULT', predictionCol="prediction")

r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
rmse = eval.evaluate(predictions, {eval.metricName: "rmse"})


print("|-------------------------------------------------------------------|")
print(f'  R2:{r2}')
print("|-------------------------------------------------------------------|")
print("|-------------------------------------------------------------------|")
print(f'  MAE:{mae}')
print("|-------------------------------------------------------------------|")
print("|-------------------------------------------------------------------|")
print(f'  RMSE:{rmse}')
print("|-------------------------------------------------------------------|")

|-------------------------------------------------------------------|
  R2:0.12493983984365464
|-------------------------------------------------------------------|
|-------------------------------------------------------------------|
  MAE:0.30703660409766403
|-------------------------------------------------------------------|
|-------------------------------------------------------------------|
  RMSE:0.385801660569706
|-------------------------------------------------------------------|


In [72]:
# 3.2.  Arbre de décision
#      --------------------

# 3.2.1. Pramétrage du modèle del'arbre de décision

dt = DecisionTreeClassifier(featuresCol='features',labelCol='DEFAULT')

pipeline = Pipeline(stages=[assembler, dt])

# 3.2.2. Entrainent  du modèle de régression linéaire

model = pipeline.fit(train_df_lr_dt)


# 3.2.3. Prédiction du modèle de régression linéaire

predictions = model.transform(test_df_lr_dt)


# 3.2.4. Evaluation du modèle de régression linéaire

eval = RegressionEvaluator(labelCol='DEFAULT', predictionCol="prediction")

r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
rmse = eval.evaluate(predictions, {eval.metricName: "rmse"})


print("|-------------------------------------------------------------------|")
print(f'  R2:{r2}')
print("|-------------------------------------------------------------------|")
print("|-------------------------------------------------------------------|")
print(f'  MAE:{mae}')
print("|-------------------------------------------------------------------|")
print("|-------------------------------------------------------------------|")
print(f'  RMSE:{rmse}')
print("|-------------------------------------------------------------------|")

|-------------------------------------------------------------------|
  R2:-0.04048879530329108
|-------------------------------------------------------------------|
|-------------------------------------------------------------------|
  MAE:0.17698142244696052
|-------------------------------------------------------------------|
|-------------------------------------------------------------------|
  RMSE:0.42069160016211465
|-------------------------------------------------------------------|


In [73]:
# 3.3.  Random forest
#      --------------------

# 3.3.1. Pramétrage du modèle de Random Forest

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'DEFAULT')

# 3.3.2. Entrainement du modèle de Random Forest

model = rf.fit(train_df)

# 3.3.3. Prédiction du modèle de Random Forest

predictions = model.transform(test_df)

# 3.3.4. Evaluation du modèle de Random Forest

eval = RegressionEvaluator(labelCol='DEFAULT', predictionCol="prediction")

r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
rmse = eval.evaluate(predictions, {eval.metricName: "rmse"})


print("|-------------------------------------------------------------------|")
print(f'  R2:{r2}')
print("|-------------------------------------------------------------------|")
print("|-------------------------------------------------------------------|")
print(f'  MAE:{mae}')
print("|-------------------------------------------------------------------|")
print("|-------------------------------------------------------------------|")
print(f'  RMSE:{rmse}')
print("|-------------------------------------------------------------------|")

|-------------------------------------------------------------------|
  R2:-0.06633944860275198
|-------------------------------------------------------------------|
|-------------------------------------------------------------------|
  MAE:0.18137847642079807
|-------------------------------------------------------------------|
|-------------------------------------------------------------------|
  RMSE:0.42588552032300664
|-------------------------------------------------------------------|


Itrération 1 des 3 algoritmes
----------------------------------------


|    | R2                    | MAE                 | RSME                |   |
|----|-----------------------|---------------------|---------------------|---|
| LR | 0.12394219711597554   | 0.3068875918214364  | 0.3889230681628588  |   |
| DT | -0.038181772980516726 | 0.179253665037761   | 0.42338359089336586 |   |
| RF | -0.0709867732419831   | 0.18491781430475343 | 0.4300207138089437  |   |


Itrération 2 des 3 algoritmes
----------------------------------------

|    | R2                    | MAE                 | RSME                |   |
|----|-----------------------|---------------------|---------------------|---|
| LR | 0.12394219711597554   | 0.3068875918214364  | 0.3889230681628588  |   |
| DT | -0.038181772980516726 | 0.179253665037761   | 0.42338359089336586 |   |
| RF | -0.0709867732419831   | 0.18491781430475343 | 0.4300207138089437  |   |

Ecarts entre 2 itérations
----------------------------------

|    | R2                    | MAE                 | RSME                |   |
|----|-----------------------|---------------------|---------------------|---|
| LR | 0.12394219711597554   | 0.3068875918214364  | 0.3889230681628588  |   |
| DT | -0.038181772980516726 | 0.179253665037761   | 0.42338359089336586 |   |
| RF | -0.0709867732419831   | 0.18491781430475343 | 0.4300207138089437  |   |


Moyenne des 2 itérations
------------------------------------

|    | R2                    | MAE                 | RSME                |   |
|----|-----------------------|---------------------|---------------------|---|
| LR | 0.12394219711597554   | 0.3068875918214364  | 0.3889230681628588  |   |
| DT | -0.038181772980516726 | 0.179253665037761   | 0.42338359089336586 |   |
| RF | -0.0709867732419831   | 0.18491781430475343 | 0.4300207138089437  |   |




Suite aux résultats des deux itérations des trois algoritmes, un cross validarion des trois métriques (`R2`, `MAE` et `RSME`) porte l'algoritme de `DT` (Decission Tree) en tête avec un `MAE`(Mean Absolute Error qui s'apparente au biais) plus proche de 0.

# 4. Que feriez-vous de plus avec ces données ? Quelque chose pour vous aider à trouver une meilleure solution ?



La validation d’un modèle de prédiction nécessite :

(i) de séparer un jeu de données initial en un jeu de données d’apprentissage et un jeu de données de validation ;

(ii) d’étalonner un modèle avec le jeu d’apprentissage ;

(iii) d’évaluer la qualité du modèle avec le jeu de données de validation en calculant les indicateurs suivants: R2, MAE, RSME, etc.


Pour s’assurer que la qualité du modèle soit bien mesurée, ces trois étapes doivent être répétées plusieurs fois. 
En effet, le jeu de données initial peut être séparé en un très grand nombre de paires de jeux de données d’apprentissage et de validation.